In [11]:
import pandas as pd, seaborn as sns, numpy as np, matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB

from sklearn.pipeline import Pipeline

%matplotlib inline

sns.set_style("darkgrid")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.linear_model import LogisticRegression, LinearRegression
import statsmodels.api as sm

/Users/roshkhadka/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/roshkhadka/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


## Cleaning up the dataset


In [12]:
df = pd.read_csv('../Data/Craigs_2_4', encoding = 'utf-8')
df.shape

(8143, 14)

In [13]:
df_cleaning = df[df["Body"].notnull()]
df_cleaning = df_cleaning[df_cleaning["Who_to"].notnull()]

In [57]:
df_cleaning.shape

(7639, 14)

In [58]:
df_cleaning.Body.isnull().sum()

0

In [59]:
df_cleaning.Who_to.unique()

array([u'm4w', u'm4m', u'w4m', u'm4t', u'w4mw', u'w4w', u't4mm', u'w4t',
       u't4m', u'm4ww', u'm4mw', u'ww4m', u't4t', u'mw4mw', u'mw4m',
       u'mw4w', u't4w', u'w4mm', u'm4mm', u'w4ww', u'mm4mw', u't4ww',
       u'mm4mm', u'ww4w', u'ww4mm', u'mm4ww'], dtype=object)

In [60]:
df_cleaning.Who_to = [i.split('4') for i in df_cleaning['Who_to']]

In [61]:
df_cleaning["From"] = 1
df_cleaning["To"] = 1

In [62]:
df_cleaning["From"] = df_cleaning.Who_to.map(lambda x: x[0])
df_cleaning["To"] = df_cleaning.Who_to.map(lambda x: x[1])

In [63]:
def checking_category(inp):
    if inp == "mm":
        return 'm'
    elif inp == "ww":
        return 'w'
    elif inp == "mw": 
        return 'm'
    elif inp == "wm":
        return 'w'
    else:
        return inp

df_cleaning["From"] = df_cleaning.From.map(checking_category)
df_cleaning["To"] = df_cleaning.To.map(checking_category)

In [64]:
df_cleaning.From.unique()

array([u'm', u'w', u't'], dtype=object)

In [65]:
# Droping the trans
df_cleaning = df_cleaning.ix[~(df_cleaning['From'] == 't')].reset_index(drop=True)

In [66]:
df_cleaning.City.unique()

array([u'austin', u'boston', u'honolulu', u'houston', u'losangeles',
       u'miami', u'newyork', u'sandiego', u'seattle', u'sfbay'], dtype=object)

## Creating a sudo code to for male = 1, coast, age and city

In [67]:
df_cleaning.shape[0]

7567

In [68]:
gay = []
for i in range(df_cleaning.shape[0]):
    if df_cleaning.From[i] == df_cleaning.To[i]:
        gay.append(1)
    else:
        gay.append(0)
        
df_cleaning['gay'] = gay
        
# city = []
# for i in range(df_cleaning.shape[0]):
#     if df_cleaning.City[i] == "austin":
#         city.append(0)
#     elif df_cleaning.City[i] == "boston":
#         city.append(1)
#     elif df_cleaning.City[i] == "honolulu":
#         city.append(2)
#     elif df_cleaning.City[i] == "houston":
#         city.append(3)
#     elif df_cleaning.City[i] == "losangeles":
#         city.append(4)
#     elif df_cleaning.City[i] == "miami":
#         city.append(5)
#     elif df_cleaning.City[i] == "newyork":
#         city.append(6)
#     elif df_cleaning.City[i] == "sandiego":
#         city.append(7)
#     elif df_cleaning.City[i] == "seattle":
#         city.append(8)
#     elif df_cleaning.City[i] == "sfbay":
#         city.append(9)
        
# coast = []        
# for i in range(df_cleaning.shape[0]):
#     if df_cleaning.City[i] in "austin":
#         coast.append(0)
#     elif df_cleaning.City[i] in "boston":
#         coast.append(0)
#     elif df_cleaning.City[i] == "honolulu":
#         coast.append(1)
#     elif df_cleaning.City[i] == "houston":
#         coast.append(0)
#     elif df_cleaning.City[i] == "losangeles":
#         coast.append(1)
#     elif df_cleaning.City[i] == "miami":
#         coast.append(0)
#     elif df_cleaning.City[i] == "newyork":
#         coast.append(1)
#     elif df_cleaning.City[i] == "sandiego":
#         coast.append(1)
#     elif df_cleaning.City[i] == "seattle":
#         coast.append(1)
#     elif df_cleaning.City[i] == "sfbay":
#         coast.append(1)



In [69]:
df_age = df[df["Age"].notnull()]

In [70]:
df_age.shape

(4335, 14)

In [71]:
df_Cleaning = df_cleaning
df_Cleaning['From_binary'] = df_Cleaning['From'].map(lambda x: 1 if x == 'm' else 0)

In [72]:
df_Cleaning = df_cleaning
df_Cleaning['From_binary'] = df_Cleaning['From'].map(lambda x: 1 if x == 'm' else 0)

## Exploring the data set for differences between male and female

## Using textacy to explore the data

### Extrating the data based on gender as a whole

In [18]:
import textacy
from textacy.extract import semistructured_statements as semistructured

In [19]:
# Extracting text from males and females individually:
male = df_Cleaning.Body.ix[~(df_Cleaning.From_binary == 0.0)].reset_index(drop = True)
female = df_Cleaning.Body.ix[~(df_Cleaning.From_binary == 1.0)].reset_index(drop = True)

NameError: name 'df_Cleaning' is not defined

In [ ]:
male_whole = []
female_whole = []
for i in range(male.shape[0]):
    male_whole.append(male[i])
for i in range(female.shape[0]):
    female_whole.append(female[i])

In [ ]:
def my_func(x):
    z = textacy.preprocess.normalize_whitespace(x)
    z = textacy.preprocess.preprocess_text(z, fix_unicode=True, lowercase=True, transliterate=False, no_urls=True, 
                                           no_emails=True, no_phone_numbers=True, no_numbers=True,
                                           no_currency_symbols=False, no_punct=True, no_contractions=False, 
                                           no_accents=True)
    doc = textacy.doc.Doc(z, lang=u'en')
    return doc
body = df_Cleaning['Body'].ix(my_func)

In [ ]:
# textacy.network.sents_to_semantic_network(df_Cleaning.Body, edge_weighting='cosine')

In [ ]:
# current = []
# for i in range(df_Cleaning.shape[0]):
#     doc = textacy.doc.Doc(df_Cleaning['Body'][i], lang = u'en')
#     z = [x for x in textacy.network.sents_to_semantic_network(df_Cleaning.Body, edge_weighting='cosine')]
#     current.append(z)

In [ ]:
# Cleaning the body section and also converting the info 
def my_func(x):
    z = textacy.preprocess.normalize_whitespace(x)
    z = textacy.preprocess.preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False, no_urls=True, 
                                           no_emails=True, no_phone_numbers=True, no_numbers=True,
                                           no_currency_symbols=False, no_punct=True, no_contractions=False, 
                                           no_accents=True)
#     doc = textacy.doc.Doc(z, lang=u'en')
    return z

df_Cleaning['Body_clean'] = df_Cleaning['Body'].map(my_func)

# body_txtacy = []
# for i in range(df_Cleaning.shape[0]):
#     df_Cleaning['Body'].map(my_func)

In [20]:
def my_func(x):
    z = textacy.preprocess.normalize_whitespace(x)
    z = textacy.preprocess.preprocess_text(z, fix_unicode=True, lowercase=True, transliterate=False, no_urls=True, 
                                           no_emails=True, no_phone_numbers=True, no_numbers=True,
                                           no_currency_symbols=False, no_punct=True, no_contractions=False, 
                                           no_accents=True)
    doc = textacy.doc.Doc(z, lang=u'en')
    return doc


In [21]:
df_Cleaning['Body_doc'] = df_Cleaning['Body'].map(my_func)

NameError: name 'df_Cleaning' is not defined

In [22]:
df_Cleaning.to_csv('df_cleaned_06.csv', encoding='utf-8')

NameError: name 'df_Cleaning' is not defined

In [23]:
# Extracting text from males and females individually:
male = df_Cleaning.Body.ix[~(df_Cleaning.From_binary == 0.0)].reset_index(drop = True)
female = df_Cleaning.Body.ix[~(df_Cleaning.From_binary == 1.0)].reset_index(drop = True)

NameError: name 'df_Cleaning' is not defined

In [17]:
df_Cleaning['Bolean'] = df_Cleaning['From'].map(lambda x: True if x == 'm' else False)


NameError: name 'df_Cleaning' is not defined

In [16]:
body = df_Cleaning.Body_doc
bol = df_Cleaning.Bolean

NameError: name 'df_Cleaning' is not defined

## Looking at unique words 2*

In [86]:
df_Cleaning

,Ids,Title,City,Geo_Tag,Where,Who_to,Age,Height,Body_type,Body,...,Has_Image,Has_Maps,URL,From,To,gay,From_binary,Body_clean,Body_doc,Bolean
0,5.915963e+09,Turning love to hate,austin,"(30.3423, -97.6673)","In the wind, along with my love","[m, w]",NaN,NaN,NaN,Imagine the effort it takes to cultivate love ...,...,False,True,http://austin.craigslist.org/mis/5915962984.html,m,w,0,1,imagine the effort it takes to cultivate love ...,"(imagine, the, effort, it, takes, to, cultivat...",True
1,5.963091e+09,teacher,austin,"(30.641818, -97.617309)",East View,"[m, w]",34.0,NaN,NaN,You are the ffa teacher for east view. I have ...,...,False,True,http://austin.craigslist.org/mis/5963091019.html,m,w,0,1,you are the ffa teacher for east view i have s...,"(you, are, the, ffa, teacher, for, east, view,...",True
2,5.962896e+09,apology to Pierre,austin,NaN,Austin,"[m, m]",59.0,NaN,NaN,I stood you up on Sunday. I apologize. I hadn'...,...,False,True,http://austin.craigslist.org/mis/5962895842.html,m,m,1,1,i stood you up on sunday i apologize i hadnt k...,"(i, stood, you, up, on, sunday, i, apologize, ...",True
3,5.961192e+09,Millie,austin,NaN,NaN,"[m, w]",NaN,NaN,NaN,I can't help but feel like a crazy ex-boyfrien...,...,False,True,http://austin.craigslist.org/mis/5961191561.html,m,w,0,1,i cant help but feel like a crazy exboyfriend ...,"(i, ca, nt, help, but, feel, like, a, crazy, e...",True
4,5.958635e+09,"""Hey old man"" you said then smiled",austin,NaN,a basketball game,"[m, w]",NaN,NaN,NaN,First your eyes so piercing and still so entic...,...,False,True,http://austin.craigslist.org/mis/5958635060.html,m,w,0,1,first your eyes so piercing and still so entic...,"(first, your, eyes, so, piercing, and, still, ...",True
5,5.961208e+09,MJ,austin,"(30.255412, -97.762357)",South Austin,"[m, w]",NaN,NaN,NaN,I'm going against better judgement here but de...,...,False,True,http://austin.craigslist.org/mis/5961207814.html,m,w,0,1,im going against better judgement here but des...,"(i, m, going, against, better, judgement, here...",True
6,5.961250e+09,"NYE at Barbarella, you work at Dell",austin,"(30.268352, -97.740276)",West campus,"[w, m]",NaN,NaN,NaN,Hi I pointed to you on the dance floor and you...,...,False,True,http://austin.craigslist.org/mis/5961249716.html,w,m,0,0,hi i pointed to you on the dance floor and you...,"(hi, i, pointed, to, you, on, the, dance, floo...",False
7,5.963361e+09,Dan,austin,NaN,NaN,"[w, m]",29.0,NaN,NaN,Hey for our date we went to pf Chang's petes ...,...,False,True,http://austin.craigslist.org/mis/5963361358.html,w,m,0,0,hey for our date we went to pf changs petes pi...,"(hey, for, our, date, we, went, to, pf, changs...",False
8,5.961451e+09,Dreamy buff guy outside of Fado,austin,"(30.2896, -97.7396)",Austin,"[m, m]",NaN,NaN,NaN,Last Saturday we crossed outside of Fado irish...,...,False,True,http://austin.craigslist.org/mis/5961450507.html,m,m,1,1,last saturday we crossed outside of fado irish...,"(last, saturday, we, crossed, outside, of, fad...",True
9,5.951413e+09,IT contractor that moved my computer,austin,"(30.3223, -97.739)",NaN,"[w, m]",NaN,NaN,NaN,I can't believe I'm even doing this but it's w...,...,False,True,http://austin.craigslist.org/mis/5951412967.html,w,m,0,0,i cant believe im even doing this but its wort...,"(i, ca, nt, believe, i, m, even, doing, this, ...",False


In [87]:
term = [x.to_bag_of_terms(ngrams=2, named_entities=True,
                  lemmatize=True, as_strings=True) for x in body]

In [88]:
t = []
for i in term:
    for key, value in i.iteritems():
        t.append([key])

    

In [89]:
type_2 = textacy.keyterms.most_discriminating_terms(t, bool_array_grp1 = bol, max_n_terms=5000, top_n_terms=80)

## Unique words *1

In [14]:
terms = [x.text.split for x in body]

NameError: name 'body' is not defined

In [15]:
textacy.keyterms.most_discriminating_terms(terms, bool_array_grp1 = bol, max_n_terms=5000, top_n_terms=80)

NameError: name 'textacy' is not defined

## Exploring the data using some scores that i extracted from textacy.readibility_Status

In [4]:
def fun(x):
    doc = textacy.doc.Doc(x, lang=u'en')
    z = textacy.text_stats.readability_stats(doc)
    return z
total = []
flesch_kincaid_grade_level = []
smog_index= []
automated_readability_index = []
n_chars = []
n_unique_words = []
n_words = []
n_syllables = []
coleman_liau_index = []
gunning_fog_index = []
flesch_readability_ease = []
n_sents = []
n_polysyllable_words = []


In [5]:
for i in range(df_Cleaning['Body'].shape[0]):
    total.append(fun(df_Cleaning['Body'][i]))

NameError: name 'df_Cleaning' is not defined

In [6]:
for i in total:
    for key, value in i.iteritems():
        if key == "n_unique_words":
            n_unique_words.append(value)
        if key == "flesch_kincaid_grade_level":
            flesch_kincaid_grade_level.append(value)
        elif key =="smog_index":
            smog_index.append(value)
        elif key == "automated_readability_index":
            automated_readability_index.append(value)
        elif key == "n_chars":
            n_chars.append(value)
        elif key == "n_words":
            n_words.append(value)
        elif key =="n_syllables":
            n_syllables.append(value)
        elif key == "coleman_liau_index":
            coleman_liau_index.append(value)
        elif key =="gunning_fog_index":
            gunning_fog_index.append(value)
        elif key == "flesch_readability_ease":
            flesch_readability_ease.append(value)
        elif key =="n_sents":
            n_sents.append(value)
        elif key == "n_polysyllable_words":
            n_polysyllable_words.append(value)

In [7]:
df_Cleaning['n_polysyllable_words'] = n_polysyllable_words
df_Cleaning["flesch_kincaid_grade_level"] = flesch_kincaid_grade_level
df_Cleaning['smog_index'] = smog_index
df_Cleaning['automated_readability_index'] = automated_readability_index
df_Cleaning['n_chars'] = n_chars
df_Cleaning['n_words'] = n_words
df_Cleaning['n_syllables'] = n_syllables
df_Cleaning['coleman_liau_index'] = coleman_liau_index
df_Cleaning['gunning_fog_index'] = gunning_fog_index
df_Cleaning['flesch_readability_ease'] = flesch_readability_ease
df_Cleaning['n_sents'] = n_sents

NameError: name 'df_Cleaning' is not defined

In [8]:
df_binary = pd.DataFrame()

NameError: name 'pd' is not defined

In [9]:
df_binary['n_polysyllable_words'] = n_polysyllable_words
df_binary["flesch_kincaid_grade_level"] = flesch_kincaid_grade_level
df_binary['smog_index'] = smog_index
df_binary['automated_readability_index'] = automated_readability_index
df_binary['n_chars'] = n_chars
df_binary['n_words'] = n_words
df_binary['n_syllables'] = n_syllables
df_binary['coleman_liau_index'] = coleman_liau_index
df_binary['gunning_fog_index'] = gunning_fog_index
df_binary['flesch_readability_ease'] = flesch_readability_ease
df_binary['n_sents'] = n_sents
df_binary['From_binary'] = df_Cleaning['From_binary']
df_binary['n_unique_words'] = n_unique_words
df_binary['gay'] = df_Cleaning.gay

NameError: name 'df_binary' is not defined

In [10]:
X = df_binary[['flesch_kincaid_grade_level',
'smog_index',
'automated_readability_index',
'n_chars',
'n_unique_words',
'n_words',
'n_syllables',
'coleman_liau_index',
'gunning_fog_index',
'flesch_readability_ease',
'n_sents','n_polysyllable_words']].values

NameError: name 'df_binary' is not defined

In [ ]:
y = df_binary['gay']

In [ ]:
continuous_variable = df_binary[['flesch_kincaid_grade_level',
'smog_index',
'automated_readability_index',
'n_chars',
'n_unique_words',
'n_words',
'n_syllables',
'coleman_liau_index',
'gunning_fog_index',
'flesch_readability_ease',
'n_sents','n_polysyllable_words','From_binary', 'gay']]

def correlation_heat_map(df):
    corrs = continuous_variable.corr()
    plt.rcParams['figure.figsize']=(20,20)
    mask = np.zeros_like(corrs, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    ax = sns.heatmap(corrs, mask=mask, annot=False)
    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=5, rotation=30)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=5, rotation=0)
    plt.show()

correlation_heat_map(continuous_variable)

In [ ]:
# X = df_binary[['flesch_kincaid_grade_level',
# 'automated_readability_index',
# 'coleman_liau_index',
# 'flesch_readability_ease']].values

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
logreg = LogisticRegressionCV()
logreg.fit(X, y)

print 'Logreg intercept:', logreg.intercept_
print 'Logreg coef(s):', logreg.coef_
print 'Logreg predicted probabilities:', logreg.predict_proba(X[0:5,:])

In [ ]:
Xc = X - X.mean()
print Xc[0:5, :]

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
Xn = ss.fit_transform(Xc)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xn, y, test_size=0.30)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
mnb = BernoulliNB()
mnb.fit(X_train, y_train)

In [ ]:
print 'baseline', 1 -np.mean(y_test)

In [ ]:
#mnb.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

#lr_lasso = LogisticRegressionCV(Cs=100, cv=5, penalty='l1', verbose=0, solver='liblinear')
#lr_lasso.fit(X_train, y_train)

lr_ridge = LogisticRegressionCV(Cs=100, cv=5, penalty='l2')
lr_ridge.fit(X_test, y_test)

In [ ]:
lr_ridge.score(X_test, y_test)

In [ ]:
lr_ridge.coef_

## Exploring the body section using Count and TfidfVectorizer

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('English')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def plot_freq(data, vectorizer="tfidf", stopwords = stop, ngram_range = (1,1)):
    
    if vectorizer == "tfidf":
        vect = TfidfVectorizer(stop_words=stop, sublinear_tf=False, use_idf=True, smooth_idf=True, ngram_range= ngram_range)
    else:
        vect = CountVectorizer(vocabulary=z[0]+z[1], max_df = 0.3, min_df = 0.1, stop_words = stop, ngram_range = ngram_range)
    
    X      =  vect.fit_transform(data)
    x_df   =  pd.DataFrame(X.toarray(), columns=vect.vocabulary_)
    tf_df  =  pd.DataFrame(x_df.mean(), columns=["freq"])
    
    title  =  "Frequency - Top 20: %s, Stopwords: %s" 
    
    tf_df.sort_values("freq").tail(40).plot(kind="barh", title=title, figsize=(10,5))

In [ ]:
plot_freq(df_Cleaning.Body, vectorizer = "CountVectorizer")
plot_freq(df_Cleaning.Body, vectorizer = "tfidf")

In [92]:
#Convertig countvects and transforming it to dataframe
from sklearn.preprocessing import StandardScaler, minmax_scale

cvt      =  CountVectorizer(stop_words= 'english', strip_accents='unicode', ngram_range=(2,4))
X_all    =  cvt.fit_transform(df_Cleaning.Body)
columns  =  cvt.get_feature_names()

In [93]:
x_df     =  pd.DataFrame(X_all.todense(), columns=columns)
tf_df    =  pd.DataFrame(x_df.sum(), columns=["freq"])
tf_df.sort_values("freq", ascending=False).head(100)

,freq
let know,594
eye contact,350
long shot,333
don know,287
just wanted,142
like know,141
say hi,134
didn want,132
don want,131
like meet,125


In [94]:
def hist_counts(X_all):

    count_df = pd.DataFrame(X_all.toarray())
    hist_counts = pd.DataFrame(minmax_scale(count_df.sum()))
    hist_counts.plot(kind="hist", bins=50, title="Histogram - All")
    hist_counts[hist_counts[0] > .1].plot(kind="hist", bins=50, title="Histogram - Counts > .1") # There are a lot of really common tokens within 10% -- filter them out
    hist_counts[hist_counts[0] < .01].plot(kind="hist", title="Histogram - Counts < .01") # look at the range of extreme commons that seem to exist below .01   

In [ ]:
count_df = pd.DataFrame(X_all.toarray())

In [ ]:
X_ = X_all
hist_counts(X_)

## Running a random forest

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

In [2]:
rf = RandomForestClassifier()

In [3]:
y = df_Cleaning.gay

NameError: name 'df_Cleaning' is not defined

In [925]:
# max_df = 0.01, min_df = 1,

In [964]:
features = type_1[0]+type_1[1]
features

[u'beautiful',
 u'cock',
 u'hit',
 u'hot',
 u'pants',
 u'black',
 u'locker',
 u'discreet',
 u'sauna',
 u'cum',
 u'pm',
 u'interested',
 u'stunning',
 u'white',
 u'shorts',
 u'dl',
 u'sucked',
 u'numbernumber',
 u'sexy',
 u'steam',
 u'latino',
 u'afternoon',
 u'lets',
 u'dick',
 u'guy',
 u'connect',
 u'nice',
 u'la',
 u'dressed',
 u'suck',
 u'meet',
 u'eye',
 u'register',
 u'sucking',
 u'muscular',
 u'wearing',
 u'cashier',
 u'shower',
 u'shopping',
 u'legs',
 u'hairy',
 u'bulge',
 u'beach',
 u'car',
 u'buddy',
 u'hmu',
 u'te',
 u'counter',
 u'mall',
 u'masculine',
 u'looking',
 u'mid',
 u'gym',
 u'jeans',
 u'checking',
 u'preciousmi',
 u'video',
 u'stats',
 u'fun',
 u'incredible',
 u'noon',
 u'rd',
 u'load',
 u'waitress',
 u'publix',
 u'enjoyed',
 u'ass',
 u'station',
 u'petite',
 u'yo',
 u'lunch',
 u'stall',
 u'sweat',
 u'butt',
 u'starbucks',
 u'bro',
 u'foods',
 u'hollywood',
 u'bay',
 u'swing',
 u'is',
 u'heart',
 u'of',
 u'feel',
 u'all',
 u'never',
 u'now',
 u'dont',
 u'my',
 u'b

In [980]:
cv = CountVectorizer(vocabulary=features, binary= True,
                     lowercase=True, strip_accents='unicode', stop_words='english', 
                     ngram_range=(1,1), max_features=2000)

In [995]:
# need to use the lowercased texts
body_vec = cv.fit_transform([' '.join(x.to_bag_of_words(lemmatize=True, as_strings=True).keys())
                             for x in body])

In [996]:
np.mean(body_vec, axis=0)

matrix([[  9.05247813e-02,   2.75510204e-02,   5.24781341e-02,
           6.25364431e-02,   0.00000000e+00,   9.11078717e-02,
           1.04956268e-02,   1.72011662e-02,   8.74635569e-03,
           8.30903790e-03,   2.74052478e-02,   6.51603499e-02,
           5.24781341e-03,   6.44314869e-02,   0.00000000e+00,
           7.14285714e-03,   0.00000000e+00,   5.69970845e-02,
           4.70845481e-02,   6.41399417e-03,   8.60058309e-03,
           2.08454810e-02,   0.00000000e+00,   1.60349854e-02,
           1.30320700e-01,   2.39067055e-02,   6.61807580e-02,
           1.09329446e-02,   1.31195335e-03,   2.82798834e-02,
           1.67784257e-01,   1.37026239e-01,   4.95626822e-03,
           0.00000000e+00,   6.41399417e-03,   0.00000000e+00,
           4.66472303e-03,   1.50145773e-02,   3.93586006e-03,
           0.00000000e+00,   4.08163265e-03,   4.37317784e-03,
           1.47230321e-02,   5.02915452e-02,   8.74635569e-03,
           1.18075802e-02,   3.93586006e-03,   7.288629

In [983]:
rf_params = {'max_depth': [2, 3, 4, 5, 6, None],
 'max_features': [None, 'sqrt', 'log2'],
 'min_samples_split': [2, 4, 8, 16, 32, 64, 128],
 'n_estimators': [200]}

In [984]:
y = df_Cleaning['gay'].values

In [985]:
rf_gs = GridSearchCV(rf, rf_params, cv=5, verbose=2, n_jobs=-1)
rf_gs.fit(body_vec, y)

Fitting 5 folds for each of 126 candidates, totalling 630 fits
[CV] max_features=None, min_samples_split=2, n_estimators=200, max_depth=2 
[CV] max_features=None, min_samples_split=2, n_estimators=200, max_depth=2 
[CV] max_features=None, min_samples_split=2, n_estimators=200, max_depth=2 
[CV] max_features=None, min_samples_split=2, n_estimators=200, max_depth=2 
[CV]  max_features=None, min_samples_split=2, n_estimators=200, max_depth=2, total=   1.9s
[CV] max_features=None, min_samples_split=2, n_estimators=200, max_depth=2 
[CV]  max_features=None, min_samples_split=2, n_estimators=200, max_depth=2, total=   1.9s
[CV]  max_features=None, min_samples_split=2, n_estimators=200, max_depth=2, total=   1.9s
[CV] max_features=None, min_samples_split=4, n_estimators=200, max_depth=2 
[CV] max_features=None, min_samples_split=4, n_estimators=200, max_depth=2 
[CV]  max_features=None, min_samples_split=2, n_estimators=200, max_depth=2, total=   1.9s
[CV] max_features=None, min_samples_split

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.7s


[CV]  max_features=None, min_samples_split=128, n_estimators=200, max_depth=2, total=   2.3s
[CV] max_features=sqrt, min_samples_split=2, n_estimators=200, max_depth=2 
[CV]  max_features=sqrt, min_samples_split=2, n_estimators=200, max_depth=2, total=   2.5s
[CV] max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=2 
[CV]  max_features=sqrt, min_samples_split=2, n_estimators=200, max_depth=2, total=   2.5s
[CV] max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=2 
[CV]  max_features=sqrt, min_samples_split=2, n_estimators=200, max_depth=2, total=   2.5s
[CV] max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=2 
[CV]  max_features=sqrt, min_samples_split=2, n_estimators=200, max_depth=2, total=   2.5s
[CV] max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=2 
[CV]  max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=2, total=   2.3s
[CV] max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min


[CV]  max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=3, total=   2.2s
[CV] max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=3 
[CV]  max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=3, total=   2.2s
[CV] max_features=sqrt, min_samples_split=32, n_estimators=200, max_depth=3 
[CV]  max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=3, total=   2.2s
[CV] max_features=sqrt, min_samples_split=32, n_estimators=200, max_depth=3 
[CV]  max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=3, total=   2.1s
[CV] max_features=sqrt, min_samples_split=32, n_estimators=200, max_depth=3 
[CV]  max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=3, total=   2.3s
[CV] max_features=sqrt, min_samples_split=32, n_estimators=200, max_depth=3 
[CV]  max_features=sqrt, min_samples_split=32, n_estimators=200, max_depth=3, total=   2.5s
[CV]  max_features=sqrt, min_samples_split=32, n_estimators=200

[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  4.1min


[CV]  max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=5, total=   2.3s
[CV] max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5 
[CV]  max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=5, total=   2.2s
[CV] max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5 
[CV]  max_features=sqrt, min_samples_split=4, n_estimators=200, max_depth=5, total=   2.1s
[CV] max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5 
[CV]  max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5, total=   2.1s
[CV] max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5 
[CV]  max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5, total=   2.0s
[CV] max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth=5 
[CV]  max_features=sqrt, min_samples_split=8, n_estimators=200, max_depth=5, total=   2.0s
[CV] max_features=sqrt, min_samples_split=16, n_estimators=200, max_depth

[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:  9.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'min_samples_split': [2, 4, 8, 16, 32, 64, 128], 'n_estimators': [200], 'max_depth': [2, 3, 4, 5, 6, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [986]:
print rf_gs.best_params_
print rf_gs.best_score_
rf_best = rf_gs.best_estimator_

{'max_features': 'log2', 'min_samples_split': 64, 'n_estimators': 200, 'max_depth': None}
0.780029154519


In [987]:
params = rf_gs.best_params_

In [810]:
# print rf_scores
# print np.mean(rf_scores), np.std(rf_scores)

[ 0.82520029  0.82520029  0.82507289  0.82567469  0.82567469]
0.825364569796 0.000257450259735


In [988]:
1- np.mean(y)

0.69096209912536444

In [989]:
rfc = RandomForestClassifier(**params)

In [990]:
Xtr, Xte, ytr, yte = train_test_split(body_vec, y, test_size=0.3)

In [991]:
rfc.fit(Xtr, ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=64, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [992]:
1. - np.mean(yte)

0.68999028182701649

In [993]:
rfc.score(Xte, yte)

0.77793974732750248

In [994]:
cross_val_score(rfc, body_vec, y, cv=5)

array([ 0.77915452,  0.7893586 ,  0.76457726,  0.77186589,  0.79737609])

In [886]:
cv = CountVectorizer(vocabulary= liss, max_df = 0.3, min_df = 0.1, binary= True,
                     lowercase=True, strip_accents='unicode', stop_words='english', 
                     ngram_range=(2,3), max_features=200) 

In [897]:
body_vec = cv.fit_transform(df_Cleaning['Body'])

In [910]:
y = df_Cleaning['From_binary'].values

In [911]:
X_train, X_test, y_train, y_test = train_test_split(body_vec, y, test_size=0.20)

In [912]:
rr = RandomForestClassifier(min_samples_split=2, n_estimators=200, max_depth=2, max_features=None)

In [913]:
rr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [914]:
print rr.score(X_test, y_test)
pd.Series(rr.predict(X_test)).value_counts()

0.822886297376


1    1372
dtype: int64